In [1]:
# Install necessary libraries
!pip install openai nest_asyncio scikit-learn --upgrade

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import openai
from openai import AsyncOpenAI
import nest_asyncio
import asyncio
import importlib.util

nest_asyncio.apply()

# Set OpenAI API key
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()

client = AsyncOpenAI(api_key=api_key)

# Async function to generate clean Python code from prompt
async def generate_code_from_prompt(prompt):
    explicit_instruction = "Use rmse = np.sqrt(mean_squared_error(y_test, y_pred)) explicitly to calculate RMSE. Provide ONLY executable Python code without explanations or markdown."
    completion = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": explicit_instruction},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return completion.choices[0].message.content.strip('` \n')

# Async function to correct buggy Python code
async def correct_code(buggy_code, error_message):
    correction_prompt = (
        "The following Python code produced this error:\n"
        f"{error_message}\n"
        f"{buggy_code}\n"
        "Correct this Python code. Explicitly use rmse = np.sqrt(mean_squared_error(y_test, y_pred)) to calculate RMSE. Return ONLY executable Python code, NO explanations or markdown."
    )
    completion = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": correction_prompt}],
        temperature=0.2
    )
    return completion.choices[0].message.content.strip('` \n')

# Execute and test CalculateRMSE() function from a given Python file
def execute_and_test(filepath):
    spec = importlib.util.spec_from_file_location("generated_module", filepath)
    module = importlib.util.module_from_spec(spec)
    try:
        spec.loader.exec_module(module)
        rmse = module.CalculateRMSE()
        rmse_numeric = float(rmse)
        print(f"Executed successfully. RMSE: {rmse_numeric:.4e}")
        return True, rmse_numeric
    except Exception as e:
        return False, str(e)

# Main asynchronous execution loop
async def main_async():
    programs_dir = "/content/drive/MyDrive/Programs"

    if not os.path.exists(programs_dir):
        print(f"Directory '{programs_dir}' not found.")
        return

    txt_files = [f for f in os.listdir(programs_dir) if f.endswith('.txt')]

    if not txt_files:
        print("No .txt prompt files found.")
        return

    for txt_file in txt_files:
        with open(os.path.join(programs_dir, txt_file), 'r', encoding='utf-8') as file:
            prompt = file.read()

        py_filename = os.path.splitext(txt_file)[0] + ".py"
        py_filepath = os.path.join(programs_dir, py_filename)

        code = await generate_code_from_prompt(prompt)

        max_trials = 5
        trial = 1

        while trial <= max_trials:
            with open(py_filepath, 'w', encoding='utf-8') as py_file:
                py_file.write(code)

            print(f"Trial {trial}: Running {py_filename}...")
            success, result = execute_and_test(py_filepath)

            if success:
                print(f"Success on trial {trial}. RMSE: {result:.4e}")
                break
            else:
                print(f"Trial {trial} error: {result}")
                code = await correct_code(code, result)
                trial += 1

        if trial > max_trials:
            print(f"Failed after {max_trials} trials.")

# Run the asynchronous main function
asyncio.run(main_async())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.1/571.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
Mounted at /content/drive
Trial 1: Running program1.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program1.py...
RMSE: 1.052461414767882e-05
R² Score: 0.4486683846007352
Executed successfully. RMSE: 1.0525e-05
Success on trial 2. RMSE: 1.0525e-05
Trial 1: Running program2.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program2.py...
Executed successfully. RMSE: 1.4333e-05
Success on trial 2. RMSE: 1.4333e-05
Trial 1: Running program3.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program3.py...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Executed successfully. RMSE: 6.1616e-02
Success on trial 2. RMSE: 6.1616e-02
Trial 1: Running program4.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program4.py...
RMSE: 1.43e-05
R² Score: -0.02
Executed successfully. RMSE: 1.4333e-05
Success on trial 2. RMSE: 1.4333e-05
Trial 1: Running program5.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program5.py...
Executed successfully. RMSE: 1.4333e-05
Success on trial 2. RMSE: 1.4333e-05
Trial 1: Running program6.py...
Trial 1 error: name 'python' is not defined
Trial 2: Running program6.py...
RMSE: 1.43e-05
R² Score: -0.02
Executed successfully. RMSE: 1.4333e-05
Success on trial 2. RMSE: 1.4333e-05
